In [2]:
import os
import json


import numpy as np
import pandas as pd 

from tqdm.auto import tqdm
from elasticsearch import Elasticsearch
from dotenv import load_dotenv


In [3]:
if os.path.isfile(".env"):
    load_dotenv()

In [4]:
ELASTICSEARCH_INDEX = os.getenv('ELASTICSEARCH_INDEX') 
ELASTICSEARCH_URL = os.getenv('ELASTICSEARCH_URL')
es_client = Elasticsearch(ELASTICSEARCH_URL)

In [5]:
def extrac_data_elasticseach(client: Elasticsearch, index: str) -> pd.DataFrame:
    """_summary_

    Args:
        client (Elasticsearch): Elastic search client
        index (str): Elasticsearch index

    Returns:
        pd.DataFrame: data
    """
    # Get the count of documents in the index
    doc_count = client.count(index=index)['count']
    # Initialize an empty list to store the documents
    docs = []
    total_docs = 0
    while True:
        # Use a scrolling search to retrieve all documents in the index
        for doc in tqdm(client.search(index=index, scroll='2m', size=1000)['hits']['hits'], total=doc_count):
            # Extract the source of the document and append it to the list
            docs.append(doc['_source'])
            # Check if there are more documents to retrieve
            if len(docs) >= doc_count:
                break
        if len(docs) >= doc_count:
            break
    # Convert the list of documents to a pandas DataFrame
    df = pd.DataFrame(docs)
    return df

In [7]:
df_es = extrac_data_elasticseach(es_client, ELASTICSEARCH_INDEX)

ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7f7c08ada480>: Failed to establish a new connection: [Errno 111] Connection refused))